# Giới thiệu

### Tổng quan

Chào mừng đã đến bài lab đầu tiên trong Khóa học VietAI. Lần này bạn sẽ học cách tương tác với hai thư viện được biết đến nhiều nhất - Numpy và TensorFlow. Bài toán chúng ta sẽ giải quyết trong buổi hôm nay là linear regression.
 
### Mục tiêu học tập

* Hiểu thêm về cách hoạt động của Numpy và Tensorflow 
* Có thể viết một thuật toán đơn giản (và debug) với `Tensorflow` và `Numpy` qua ví dụ về hồi quy tuyến tính.
* Hiểu thêm về hồi quy tuyến tính.

# Giới thiệu

Bài toán lần này, chúng ta sẽ cùng nhau dự đoán giá nhà tại một thành phố Boston ở Mỹ với kích thước mẫu với 506 giá trị với 13 thuộc tính như sau

```
- CRIM per capita crime rate by town
- ZN proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS proportion of non-retail business acres per town
- CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- NOX nitric oxides concentration (parts per 10 million)
- RM average number of rooms per dwelling
- AGE proportion of owner-occupied units built prior to 1940
- DIS weighted distances to five Boston employment centres
- RAD index of accessibility to radial highways
- TAX full-value property-tax rate per $10,000
- PTRATIO pupil-teacher ratio by town
- B 1000(Bk - 0.63)ˆ2 where Bk is the proportion of blacks by town
- LSTAT % lower status of the population
- MEDV Median value of owner-occupied homes in $1000's
```

Với thông tin này, ta sẽ cùng nhau tiến hành xây dựng một hàm giả thuyết theo mô hình hồi quy tuyến tính dựa trên `numpy` và `tensorflow` để đưa ra giá nhà dựa trên ít nhất là 2 thuộc tính.

### I/ Tìm nghiệm bài toán Linear Regression bằng `Numpy`

#### Nhập thư viện

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Đọc dữ liệu

In [3]:
from utils_function import load_Boston_housing_data

train_X, test_X, train_Y, test_Y = load_Boston_housing_data(feature_ind = [2,5])

Selected original features are ['INDUS' 'RM']


Vừa rồi, ta vừa mới tải tập dữ liệu xuống và chiết ra thành 4 tập. 

Trong đó, `train_X` và `train_Y` được dùng để xây dựng mô hình. Còn `test_X`, `test_Y` được giữ lại và chỉ được dùng để đánh giá độ tốt của mô hình ở phút cuối. 

Tỉ lệ được chia ra giữa các phần xây dựng và kiểm chứng là 1:4 Sau này, chúng ta sẽ học kỹ hơn về phần này nên phần đánh giá độ tốt của mô hình mình chỉ dừng lại ở mức là thực thi câu hàm đánh giá.

Để đơn giản, dữ liệu được lấy vào chỉ gồm 2 đặc tính

```
    - INDUS proportion of non-retail business acres per town
    - RM average number of rooms per dwelling
```

Tạm dịch

```
    - INDUS: tỷ lệ diện tích không dùng cho kinh doanh bán lẻ mỗi thị trấn
    - RM: số phòng trung bình mỗi căn hộ/ nhà
```

Bạn có thể lấy thêm các đặc tính khác qua việc thiết lập lại hàm `load_data` như sau

In [4]:
load_Boston_housing_data?

Signature:
load_Boston_housing_data(
    test_ratio=0.2,
    feature_ind=None,
    random_state=0,
    print_info=False,
)
Docstring:
Load Boston Dataset from Sklearn.

Args:
  test_ratio(float)       : a proportion between train set and test set. Default = 0.2
  feature_ind(list(int))  : a list of index feature to be extracted from the original data set. No indication
                            means that all feature is chosen.
  random_state(int)       : a seed value for shuffling between values
  print_info(boolean)     : True if print the information of data set. Default: False
  
Returns:
  a tuple of four np.array data sets train_data, test_data, train_targets, test_targets
  
File:      /media/prime/PRIME/Emage/vietAI/Tutotial_Linear_Regression/utils_function.py
Type:      function


Để hiểu thêm về tập dữ liệu ta có thể làm thêm một số minh họa như sau

#### Minh họa dữ liệu

In [ ]:
from utils_function import scatter_plot

scatter_plot(train_X[:, 0], train_Y,\
            title="Industrialness vs Med House Price",\
            x_label = "Industrialness",\
            y_label = "Med House Price")


scatter_plot(train_X[:, 1], train_Y,\
            title="Avg Num Rooms vs Med House Price",\
            x_label = "Avg Num Rooms",\
            y_label = "Med House Price")

Hiên thời, vecto của ta đang có dạng như sau:

In [ ]:
print("Size of data X train matrix %s"  % str(train_X.shape))
print("Size of data Y train array %s"  % str(train_Y.shape))
print("Size of data X test matrix %s"  % str(test_X.shape))
print("Size of data Y test array %s"  % str(test_Y.shape))

#### Bài 1. Viết hàm cost function
Nhắc lại, từ công thức tính **mean squared error**, ta có:
$$\mathcal{E}(\hat{y}, y) = \frac{1}{2M} \sum_{i=1}^M (\hat{y}^{(i)}-y^{(i)})^2 $$
Khai triển, kết quả có là
$$\mathcal{E}(\hat{y}, y) = \frac{1}{2M} \sum_{i=1}^M (w_1 x_1^{(i)} + w_2 x_2^{(i)} + b - y^{(i)})^2 $$

#### 1.1. Viết chương trình

In [12]:
def np_cost(w, b, X, y):
    '''
    Evaluate the cost function in a non-vectorized manner for 
    inputs `X` and targets `y`, at weights `w` and `b`.
    
    X: 2D array dataset that has (M, n) dimension. 
    y: list of targets that has (M, 1) dimension.
    w: list of weights that has (n, 1) dimension
    b: a scalar bias.
    
    Return a scalar cost value of `w`, `b`.
    '''
    
    costs = 0
    M = len(y)
    for i in range(len(y)):
        
        # TODO: complete below expression to calculate cost function
        y_hat = None # hypothesis
        costs += None # sum cost of each data point.
        
    return costs / (2 * M)

#### 1.2 Chạy thử chương trình 

In [ ]:
np_cost([3, 5], 20, train_X, train_Y) # 2256.1627893564355

In [ ]:
np_cost([3, 5], 0, train_X, train_Y) # 1210.8142745049508

#### Bài 2. Vector hóa hàm cost

Ngoài ra, ta còn có thể viết lại công thức trên như sau:

$$\mathcal{E}(\hat{y}, y) = \frac{1}{2M} \| \bf{X} \bf{w} + b  - \bf{y} \| ^2$$ 

#### 2.1 Viết chương trình

In [13]:
def np_cost_vectorized(w, b, X, y):
    '''
    Evaluate the cost function in a vectorized manner for 
    inputs `X` and targets `t`, at weights `w` and `b`.
    
    X: dataset matrix has (M, n) dimension. 
    y: targets vector has (M, 1) dimension.
    w: weights vector has (n, 1) dimension
    b: a scalar bias.
    
    Return a scalar cost value of `w`, `b`.
    '''
    
    #TODO: Complete the following expression
    M = None # number of dataset
    w = None # convert to numpy array
    y_hat = None # hypothesis
    
    return np.sum((y_hat - y)**2) / (2.0 * M)

#### 2.2 Chạy thử chương trình

In [ ]:
np_cost_vectorized([3, 5], 20, train_X, train_Y) # 2256.1627893564355

In [ ]:
np_cost_vectorized([3, 5], 0, train_X, train_Y) # 1210.8142745049508

### Tìm nghiệm chính xác

#### Bài 3. Tính nghiệm chính xác

Để chuyển qua phép tính ma trận có dạng như sau $X *w + b$ ta cần bổ sung thêm một chiều cho hệ số tự do ở phần cuối của mỗi dòng trong ma trận. 

**Ví dụ** Lấy dòng đầu tiên của ma trận hiện thời, ta có: `array([ 6.2  ,  6.951 ])`   $\,\to\,$  `array([ 6.2  ,  6.951,  1.   ])`

#### 3. 1 Tái tạo vecto cho tập train

In [ ]:
# TODO: Add one at the end of each row in train_X by np.concatenate
train_X_new = None

In [ ]:
train_X_new[0]

#### 3.2 Viết chương trình tính nghiệm
 
$$W = \left(X^{\rm T}X\right)^{-1} \left(X^{\rm T}y \right) =  A * c $$

In [15]:
def np_solve_exactly(X, y):
    '''
    Solve linear regression exactly. (fully vectorized)
    
    Given `X` - Mxn matrix of inputs
          `t` - target outputs
    Return the optimal weights as a n-dimensional vector

    '''
    
    #TODO: Complete the below followed the above expressions
    A = None
    c = None
    
    return np.dot(np.linalg.inv(A), c)

#### 3.3 Chạy thử chương trình

In [ ]:
opt_w_exactly = np_solve_exactly(train_X_new, train_Y)
print("W:",opt_w_exactly) # [ -0.37785864   7.83254633 -22.54414787]
print("Cost:", np_cost_vectorized(opt_w_exactly[:2], opt_w_exactly[2], train_X, train_Y)) # 18.501517485624333

### Tìm nghiệm xấp xỉ bằng `Numpy`


#### Bài 4. Véc-tơ hóa hàm tính grad

Với $i = 1...m $ và $j = 1...n$ xét

$$ \frac{\partial \mathcal{E}}{\partial w_j} = \frac{1}{M}\sum_i x_j^{(i)}(\hat{y}^{(i)}-y^{(i)}) $$



#### 4.1 Viết chương trình

In [18]:
def np_grad_fn(weights, X, y):
    '''
    Given `weights` - a current "Guess" of what our weights should be
          `X` - matrix of shape (M,n) of input features
          `t` - target y values
    Return gradient of each weight evaluated at the current value
    '''
    
    #TODO: Complete the below followed the above expressions
    M, n = None
    y_hat = None
    error = None
    
    return np.dot(np.transpose(X), error) / float(M)

#### Bài  5. Tính dựa trên Gradient Descent

Xấp xỉ giá trị $W$ như sau

$$ W := W - \alpha * \left( h_\theta(x^{(i)}) - y^{(i)} \right)x_j^{(i)}$$

#### 5.1 Viết chương trình

In [17]:
def np_solve_via_gradient_descent(X, y, print_every=5000,
                               niter=100000, alpha=0.005):
    '''
    Given `X` - matrix of shape (M,n) of input features
          `y` - target y values
    Solves for linear regression weights.
    Return weights after `niter` iterations.
    '''
    M, n = np.shape(X)
    # initialize all the weights to zeros
    w = np.zeros([n])
    for k in range(niter):
        
        #TODO: Complete the below followed the above expressions
        dw = None
        w = None
        
        if k % print_every == 0:
            print('Weight after %d iteration: %s' % (k, str(w)))
    return w

#### 5.2 Chạy thử chương trình

In [ ]:
opt_w = np_solve_via_gradient_descent(train_X_new, train_Y)

In [ ]:
print("Training cost:", np_cost_vectorized(opt_w[:2], opt_w[2], train_X, train_Y)) # 18.502043170574055

In [ ]:
print("Testing cost:", np_cost_vectorized(opt_w[:2], opt_w[2], test_X, test_Y)) # 24.051240417619557